In [2]:
from utils import EyeTrackingDataLoader
from transformers import AutoTokenizer

### Script parameters
- Language: supported 'it' and 'en'
- Rollout: if to apply rollout to ValueZeroing
- Aggregation method: how to compute the attention of a word composed of multiple sub-tokens. Supported values:
	- sum: sum attention values of all sub-tokens
	- max: take the maximum attention value among sub-tokens
	- mean: average the attention values of the sub-tokens
	- first: take the attention value of the first sub-token
- Layer: the layer from which to extract attention

In [3]:
language = 'en'
rollout = True
aggregation_method = 'sum'
layer = -1

In [4]:
eye_tracking_data_dir = f'../augmenting_nlms_meco_data/{language}'
dl = EyeTrackingDataLoader(eye_tracking_data_dir)
sentences_df = dl.load_sentences()

In [5]:
sentences_df

,sent_id,sentence
0,1_1,"[In, ancient, Roman, religion, and, myth,, Jan..."
1,1_2,"[He, has, a, double, nature, and, is, usually,..."
2,1_3,"[Janus, presided, over, the, beginning, and, e..."
3,1_4,"[The, doors, of, his, temple, were, open, in, ..."
4,1_5,"[As, the, god, of, gates,, he, was, also, asso..."
...,...,...
96,12_4,"[The, registration, identifier, is, a, series,..."
97,12_5,"[In, some, countries,, the, identifier, is, un..."
98,12_6,"[France, was, the, first, country, to, introdu..."
99,12_7,"[Early, twentieth, century, plates, varied, in..."


In [6]:
model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
subword_prefic = '▁'